In [1]:
import time
import argparse
import logging
import torch
import os

from utils_copy import load_config, setup_logging, log_hyperparameters, seed_everything, activate_kaggle_user # dont touch
from load_data import load_train, normalization

from train import train_model
from test_new import test_model

In [2]:
if __name__ == "__main__":
    activate_kaggle_user('limkim')
    parser = argparse.ArgumentParser(description='Main script for training the model.')
    parser.add_argument('--config', type=str, default='hyper.yaml', help='Path to the config file..')
    args, unknown = parser.parse_known_args()

    config = load_config(args.config)
    setup_logging('training.log')
    
    logging.info("-----------------------------------------------------------------------------")
    logging.info(f"Hyperparameters:")
    log_hyperparameters(config)

    ts = time.time()
    
    x_train, y_train, FEAT_COLS, TARGET_COLS = load_train(ts) 
    x_train, y_train, mx, sx, my, sy = normalization(x_train, y_train) # x_train:[_, 556(feat_cols)], y_train:[_, 368(target_cols)]
    
    seed_everything()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("am i using gpu? :", device)

    if not os.path.exists(config['MODEL_PATH']):
        train_model(config, x_train, y_train, FEAT_COLS, TARGET_COLS, device, ts)
        test_model(config, mx, sx, my, sy, FEAT_COLS, TARGET_COLS, device)
    else:
        test_model(config, mx, sx, my, sy, FEAT_COLS, TARGET_COLS, device)


2024-06-24 20:57:57,387 - -----------------------------------------------------------------------------
2024-06-24 20:57:57,387 - Hyperparameters:
2024-06-24 20:57:57,388 - BATCH_SIZE: 1024
2024-06-24 20:57:57,388 - MIN_STD: 1e-8
2024-06-24 20:57:57,389 - SCHEDULER_PATIENCE: 3
2024-06-24 20:57:57,389 - SCHEDULER_FACTOR: 0.316
2024-06-24 20:57:57,390 - EPOCHS: 100
2024-06-24 20:57:57,390 - PATIENCE: 6
2024-06-24 20:57:57,391 - PRINT_FREQ: 2100
2024-06-24 20:57:57,391 - MODEL_PATH: FFNN_100.pth
2024-06-24 20:57:57,391 - LEARNING_RATE: 0.001
2024-06-24 20:57:57,392 - WEIGHT_DECAY: 0.01


Activated Kaggle user: limkim
time to read dataset: 0:00:15
am i using gpu? : cuda:0
Submission file FFNN_100.csv created.


In [4]:
command = f'kaggle competitions submit -c leap-atmospheric-physics-ai-climsim -f {config["MODEL_PATH"]} -m "{config["MODEL_PATH"]}"'

os.system(command)

100%|██████████| 75.0M/75.0M [00:07<00:00, 10.3MB/s]


Successfully submitted to LEAP - Atmospheric Physics using AI (ClimSim)

0